# ⚖️ AI Ethics, Bias & Fairness

**Author**: Data Science Master System  
**Difficulty**: ⭐⭐⭐ Advanced  
**Time**: 60 minutes  
**Prerequisites**: ML Fundamentals

## Learning Objectives
- Understand bias in ML systems
- Implement fairness metrics
- Apply bias mitigation techniques
- Build explainable AI systems

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

np.random.seed(42)

## 1. Types of Bias

In [ ]:
bias_types = '''
⚖️ TYPES OF BIAS IN ML

1. DATA BIAS
   - Historical bias: Past discrimination in data
   - Representation bias: Underrepresented groups
   - Measurement bias: Different quality across groups

2. ALGORITHMIC BIAS
   - Optimization bias: Metric favors majority
   - Aggregation bias: One model for all groups

3. DEPLOYMENT BIAS
   - Population shift: Different users than training
   - Feedback loops: Predictions affect future data

REAL-WORLD EXAMPLES:
- Hiring algorithms favoring males
- Facial recognition failing on darker skin
- Loan approvals disadvantaging minorities
- Recidivism prediction with racial disparities
'''
print(bias_types)

## 2. Fairness Metrics

In [ ]:
def demographic_parity(y_pred, protected_attr):
    """Selection rate should be equal across groups."""
    groups = np.unique(protected_attr)
    rates = {}
    for g in groups:
        mask = protected_attr == g
        rates[g] = y_pred[mask].mean()
    return rates

def equalized_odds(y_true, y_pred, protected_attr):
    """TPR and FPR should be equal across groups."""
    groups = np.unique(protected_attr)
    metrics = {}
    
    for g in groups:
        mask = protected_attr == g
        cm = confusion_matrix(y_true[mask], y_pred[mask], labels=[0, 1])
        tn, fp, fn, tp = cm.ravel() if cm.size == 4 else (0, 0, 0, 0)
        
        tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
        fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
        metrics[g] = {'TPR': tpr, 'FPR': fpr}
    
    return metrics

def disparate_impact(y_pred, protected_attr, privileged=1):
    """Ratio of selection rates (should be > 0.8)."""
    rates = demographic_parity(y_pred, protected_attr)
    groups = list(rates.keys())
    
    if len(groups) < 2:
        return 1.0
    
    priv_rate = rates[privileged]
    unpriv_rates = [rates[g] for g in groups if g != privileged]
    
    if priv_rate == 0:
        return 0
    
    return min(unpriv_rates) / priv_rate

print("📊 Fairness Metrics:")
print("  - Demographic Parity: Equal selection rates")
print("  - Equalized Odds: Equal TPR and FPR")
print("  - Disparate Impact: Ratio > 0.8 (80% rule)")

## 3. Bias Detection Example

In [ ]:
# Create biased dataset
n = 1000
protected = np.random.choice([0, 1], n, p=[0.3, 0.7])  # Group 0 is minority
features = np.random.randn(n, 5)

# Bias: Group 0 less likely to get positive outcome
score = features[:, 0] + features[:, 1] + 0.5 * protected  # Protected group advantage
labels = (score > np.median(score)).astype(int)

# Train model
X_train, X_test, y_train, y_test, p_train, p_test = train_test_split(
    features, labels, protected, test_size=0.2
)

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Evaluate fairness
print("📊 Fairness Analysis:")
print(f"\nDemographic Parity: {demographic_parity(y_pred, p_test)}")
print(f"\nEqualized Odds: {equalized_odds(y_test, y_pred, p_test)}")
print(f"\nDisparate Impact: {disparate_impact(y_pred, p_test, privileged=1):.3f}")

## 4. Bias Mitigation

In [ ]:
mitigation_strategies = pd.DataFrame({
    'Stage': ['Pre-processing', 'Pre-processing', 'In-processing', 'Post-processing'],
    'Technique': ['Reweighting', 'Resampling', 'Adversarial Debiasing', 'Threshold Adjustment'],
    'Description': [
        'Assign weights to balance groups',
        'Over/undersample to balance',
        'Train adversary to remove bias',
        'Different thresholds per group'
    ],
    'Pros': [
        'Simple',
        'Easy to implement',
        'Learned debiasing',
        'No retraining'
    ]
})

print("🔧 Bias Mitigation Strategies:")
display(mitigation_strategies)

## 5. Explainable AI (XAI)

In [ ]:
xai_code = '''
# SHAP for model explanations
import shap

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

# Global feature importance
shap.summary_plot(shap_values, X_test)

# Local explanation for single prediction
shap.force_plot(explainer.expected_value, shap_values[0], X_test[0])

# LIME for any model
from lime.lime_tabular import LimeTabularExplainer

explainer = LimeTabularExplainer(X_train, feature_names=feature_names)
exp = explainer.explain_instance(X_test[0], model.predict_proba)
exp.show_in_notebook()
'''
print("📋 Explainable AI with SHAP & LIME:")
print(xai_code)

## 6. Ethical AI Framework

In [ ]:
framework = '''
📋 ETHICAL AI CHECKLIST

□ DATA STAGE
  □ Audit data for historical bias
  □ Check representation across groups
  □ Document data sources and limitations

□ DEVELOPMENT STAGE
  □ Define fairness metrics for the use case
  □ Test model on subgroups
  □ Document model behavior and limitations

□ DEPLOYMENT STAGE
  □ Monitor for drift in fairness metrics
  □ Provide explanations for decisions
  □ Enable human override for high-stakes decisions

□ GOVERNANCE
  □ Clear accountability structure
  □ Regular bias audits
  □ User feedback mechanisms
'''
print(framework)

## 🎯 Key Takeaways
1. Bias can enter at any stage of ML pipeline
2. Multiple fairness metrics exist - choose based on context
3. Mitigation strategies: pre/in/post-processing
4. XAI (SHAP, LIME) provides transparency
5. Ethical AI requires ongoing monitoring

**Libraries**: Fairlearn, AIF360, SHAP, LIME